# Sleep Quality Predictors.Model Classification

In [1]:
import pandas as pd
df=pd.read_csv('sleep_cycle_productivity.csv')
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder,LabelEncoder,StandardScaler
from sklearn.preprocessing import MinMaxScaler,StandardScaler,RobustScaler
from sklearn.model_selection import train_test_split,KFold, cross_val_score

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 15 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Date                           5000 non-null   object 
 1   Person_ID                      5000 non-null   int64  
 2   Age                            5000 non-null   int64  
 3   Gender                         5000 non-null   object 
 4   Sleep Start Time               5000 non-null   float64
 5   Sleep End Time                 5000 non-null   float64
 6   Total Sleep Hours              5000 non-null   float64
 7   Sleep Quality                  5000 non-null   int64  
 8   Exercise (mins/day)            5000 non-null   int64  
 9   Caffeine Intake (mg)           5000 non-null   int64  
 10  Screen Time Before Bed (mins)  5000 non-null   int64  
 11  Work Hours (hrs/day)           5000 non-null   float64
 12  Productivity Score             5000 non-null   i

In [3]:
df.head()

,Date,Person_ID,Age,Gender,Sleep Start Time,Sleep End Time,Total Sleep Hours,Sleep Quality,Exercise (mins/day),Caffeine Intake (mg),Screen Time Before Bed (mins),Work Hours (hrs/day),Productivity Score,Mood Score,Stress Level
0,2024-04-12,1860,32,Other,23.33,4.61,5.28,3,86,87,116,8.808920,8,3,6
1,2024-11-04,1769,41,Female,21.02,2.43,5.41,5,32,21,88,6.329833,10,3,7
2,2024-08-31,2528,20,Male,22.10,3.45,5.35,7,17,88,59,8.506306,10,9,10
3,2024-02-22,8041,37,Other,23.10,6.65,7.55,8,46,34,80,6.070240,8,4,2
4,2024-02-23,4843,46,Other,21.42,4.17,6.75,10,61,269,94,11.374994,8,7,9


In [4]:
df.isnull().sum()         # tushib qolgan qiymatlar yo'q

Date                             0
Person_ID                        0
Age                              0
Gender                           0
Sleep Start Time                 0
Sleep End Time                   0
Total Sleep Hours                0
Sleep Quality                    0
Exercise (mins/day)              0
Caffeine Intake (mg)             0
Screen Time Before Bed (mins)    0
Work Hours (hrs/day)             0
Productivity Score               0
Mood Score                       0
Stress Level                     0
dtype: int64

In [5]:
print(df.columns)

Index(['Date', 'Person_ID', 'Age', 'Gender', 'Sleep Start Time',
       'Sleep End Time', 'Total Sleep Hours', 'Sleep Quality',
       'Exercise (mins/day)', 'Caffeine Intake (mg)',
       'Screen Time Before Bed (mins)', 'Work Hours (hrs/day)',
       'Productivity Score', 'Mood Score', 'Stress Level'],
      dtype='object')


In [6]:
df.drop(columns=['Date'], inplace=True)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 14 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Person_ID                      5000 non-null   int64  
 1   Age                            5000 non-null   int64  
 2   Gender                         5000 non-null   object 
 3   Sleep Start Time               5000 non-null   float64
 4   Sleep End Time                 5000 non-null   float64
 5   Total Sleep Hours              5000 non-null   float64
 6   Sleep Quality                  5000 non-null   int64  
 7   Exercise (mins/day)            5000 non-null   int64  
 8   Caffeine Intake (mg)           5000 non-null   int64  
 9   Screen Time Before Bed (mins)  5000 non-null   int64  
 10  Work Hours (hrs/day)           5000 non-null   float64
 11  Productivity Score             5000 non-null   int64  
 12  Mood Score                     5000 non-null   i

In [8]:
df['Gender'].value_counts()

Gender
Male      1718
Female    1675
Other     1607
Name: count, dtype: int64

In [9]:
# Encoding
label=LabelEncoder()

In [10]:
df['Gender']=label.fit_transform(df['Gender'])

In [11]:
df.head()

,Person_ID,Age,Gender,Sleep Start Time,Sleep End Time,Total Sleep Hours,Sleep Quality,Exercise (mins/day),Caffeine Intake (mg),Screen Time Before Bed (mins),Work Hours (hrs/day),Productivity Score,Mood Score,Stress Level
0,1860,32,2,23.33,4.61,5.28,3,86,87,116,8.808920,8,3,6
1,1769,41,0,21.02,2.43,5.41,5,32,21,88,6.329833,10,3,7
2,2528,20,1,22.10,3.45,5.35,7,17,88,59,8.506306,10,9,10
3,8041,37,2,23.10,6.65,7.55,8,46,34,80,6.070240,8,4,2
4,4843,46,2,21.42,4.17,6.75,10,61,269,94,11.374994,8,7,9


In [12]:
# Feature Engineering
import numpy as np

# Nap Duration
df['Nap Duration'] = (8 - df['Total Sleep Hours']) * 10 + df['Stress Level'] * 5 - df['Mood Score'] * 3 - df['Work Hours (hrs/day)'] * 2
df['Nap Duration'] = np.maximum(df['Nap Duration'], 0)  # Manfiy qiymatlarni 0ga olib kelish

# Room Environment
df['Room Environment'] = df['Sleep Quality'] * 10 - df['Screen Time Before Bed (mins)'] * 0.3 - df['Caffeine Intake (mg)'] * 0.05
df['Room Environment'] = df['Room Environment'].clip(0, 100)  # 0-100 oraliqqa cheklash

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 16 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Person_ID                      5000 non-null   int64  
 1   Age                            5000 non-null   int64  
 2   Gender                         5000 non-null   int64  
 3   Sleep Start Time               5000 non-null   float64
 4   Sleep End Time                 5000 non-null   float64
 5   Total Sleep Hours              5000 non-null   float64
 6   Sleep Quality                  5000 non-null   int64  
 7   Exercise (mins/day)            5000 non-null   int64  
 8   Caffeine Intake (mg)           5000 non-null   int64  
 9   Screen Time Before Bed (mins)  5000 non-null   int64  
 10  Work Hours (hrs/day)           5000 non-null   float64
 11  Productivity Score             5000 non-null   int64  
 12  Mood Score                     5000 non-null   i

In [14]:
df[['Nap Duration','Room Environment']].describe()

,Nap Duration,Room Environment
count,5000.000000,5000.000000
mean,12.472526,26.028710
std,14.928831,25.749406
min,0.000000,0.000000
25%,0.000000,0.000000
50%,6.348696,20.600000
75%,22.051567,45.662500
max,69.575729,98.900000


In [15]:
df.head()

,Person_ID,Age,Gender,Sleep Start Time,Sleep End Time,Total Sleep Hours,Sleep Quality,Exercise (mins/day),Caffeine Intake (mg),Screen Time Before Bed (mins),Work Hours (hrs/day),Productivity Score,Mood Score,Stress Level,Nap Duration,Room Environment
0,1860,32,2,23.33,4.61,5.28,3,86,87,116,8.808920,8,3,6,30.582160,0.00
1,1769,41,0,21.02,2.43,5.41,5,32,21,88,6.329833,10,3,7,39.240334,22.55
2,2528,20,1,22.10,3.45,5.35,7,17,88,59,8.506306,10,9,10,32.487389,47.90
3,8041,37,2,23.10,6.65,7.55,8,46,34,80,6.070240,8,4,2,0.000000,54.30
4,4843,46,2,21.42,4.17,6.75,10,61,269,94,11.374994,8,7,9,13.750012,58.35


In [16]:
# Scaling
scaler=MinMaxScaler()


In [17]:
numerical_columns=df.select_dtypes(include='number').columns

In [18]:
numerical_columns

Index(['Person_ID', 'Age', 'Gender', 'Sleep Start Time', 'Sleep End Time',
       'Total Sleep Hours', 'Sleep Quality', 'Exercise (mins/day)',
       'Caffeine Intake (mg)', 'Screen Time Before Bed (mins)',
       'Work Hours (hrs/day)', 'Productivity Score', 'Mood Score',
       'Stress Level', 'Nap Duration', 'Room Environment'],
      dtype='object')

In [19]:
df[numerical_columns]=scaler.fit_transform(df[numerical_columns])

In [20]:
df.head()

,Person_ID,Age,Gender,Sleep Start Time,Sleep End Time,Total Sleep Hours,Sleep Quality,Exercise (mins/day),Caffeine Intake (mg),Screen Time Before Bed (mins),Work Hours (hrs/day),Productivity Score,Mood Score,Stress Level,Nap Duration,Room Environment
0,0.095577,0.341463,1.0,0.836683,0.457111,0.156,0.222222,0.966292,0.290970,0.648045,0.601176,0.777778,0.222222,0.555556,0.439552,0.000000
1,0.085463,0.560976,0.0,0.256281,0.211061,0.182,0.444444,0.359551,0.070234,0.491620,0.291254,1.000000,0.222222,0.666667,0.563995,0.228008
2,0.169816,0.048780,0.5,0.527638,0.326185,0.170,0.666667,0.191011,0.294314,0.329609,0.563345,1.000000,0.888889,1.000000,0.466936,0.484328
3,0.782507,0.463415,1.0,0.778894,0.687359,0.610,0.777778,0.516854,0.113712,0.446927,0.258802,0.777778,0.333333,0.111111,0.000000,0.549039
4,0.427095,0.682927,1.0,0.356784,0.407449,0.450,1.000000,0.685393,0.899666,0.525140,0.921973,0.777778,0.666667,0.888889,0.197627,0.589990
